In [7]:
import yfinance as yf
import pandas as pd


# Define the ticker symbol for the stock you want to trade
ticker_symbol = 'TCS'

# Define the start and end dates for the data
start_date = "2020-01-01"
end_date = "2022-03-20"

# Fetch the data from Yahoo Finance API
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Print the first 5 rows of the data
print(stock_data.head())

# Define the period for calculating the moving average and standard deviation
period = 20

# Calculate the moving average
stock_data['MA'] = stock_data['Close'].rolling(window=period).mean()

# Calculate the standard deviation
stock_data['STD'] = stock_data['Close'].rolling(window=period).std()

# Print the first 5 rows of the data with moving average and standard deviation
print(stock_data.head())

[*********************100%***********************]  1 of 1 completed
            Open  High   Low  Close  Adj Close  Volume
Date                                                  
2020-01-02  4.29  4.37  4.17   4.27   3.991522  142800
2020-01-03  4.22  4.32  4.18   4.28   4.000870  141500
2020-01-06  4.23  4.46  4.20   4.45   4.159782  118500
2020-01-07  4.44  4.45  4.31   4.39   4.103695  104500
2020-01-08  4.36  4.45  4.33   4.34   4.056957  213400
            Open  High   Low  Close  Adj Close  Volume  MA  STD
Date                                                           
2020-01-02  4.29  4.37  4.17   4.27   3.991522  142800 NaN  NaN
2020-01-03  4.22  4.32  4.18   4.28   4.000870  141500 NaN  NaN
2020-01-06  4.23  4.46  4.20   4.45   4.159782  118500 NaN  NaN
2020-01-07  4.44  4.45  4.31   4.39   4.103695  104500 NaN  NaN
2020-01-08  4.36  4.45  4.33   4.34   4.056957  213400 NaN  NaN


In [8]:

# Define the number of standard deviations for upper and lower bounds
num_std = 2

# Calculate the upper and lower bounds
stock_data['Upper'] = stock_data['MA'] + (stock_data['STD'] * num_std)
stock_data['Lower'] = stock_data['MA'] - (stock_data['STD'] * num_std)

# Print the first 5 rows of the data with moving average, standard deviation, upper and lower bounds
print(stock_data.head())



            Open  High   Low  Close  Adj Close  Volume  MA  STD  Upper  Lower
Date                                                                         
2020-01-02  4.29  4.37  4.17   4.27   3.991522  142800 NaN  NaN    NaN    NaN
2020-01-03  4.22  4.32  4.18   4.28   4.000870  141500 NaN  NaN    NaN    NaN
2020-01-06  4.23  4.46  4.20   4.45   4.159782  118500 NaN  NaN    NaN    NaN
2020-01-07  4.44  4.45  4.31   4.39   4.103695  104500 NaN  NaN    NaN    NaN
2020-01-08  4.36  4.45  4.33   4.34   4.056957  213400 NaN  NaN    NaN    NaN


In [13]:
# Define the initial capital for trading
initial_capital = 10000

# Define the number of shares to trade
num_shares = 10

# Initialize the position
position = None

# Initialize the list of trades
trades = []

# Loop through each row in the stock data
for i, row in stock_data.iterrows():

    # Buy the stock if it crosses the lower bound
    if row['Close'] < row['Lower'] and position != 'long':
        # Calculate the number of shares to buy
        num_buy_shares = int(initial_capital / (num_shares * row['Close']))
        # Calculate the total cost of buying the shares
        total_buy_cost = num_buy_shares * num_shares * row['Close']
        # Subtract the total cost from the initial capital
        initial_capital -= total_buy_cost
        # Update the position
        position = 'long'
        # Add the trade to the list of trades
        trades.append(('buy', num_buy_shares, row['Close'], total_buy_cost))
    
    # Short the stock if it crosses the upper bound
    elif row['Close'] > row['Upper'] and position != 'short':
        # Calculate the number of shares to short
        num_short_shares = int(initial_capital / (num_shares * row['Close']))
        # Calculate the total proceeds from shorting the shares
        total_short_proceeds = num_short_shares * num_shares * row['Close']
        # Add the total proceeds to the initial capital
        initial_capital += total_short_proceeds
        # Update the position
        position = 'short'
        # Add the trade to the list of trades
        trades.append(('short', num_short_shares, row['Close'], total_short_proceeds))
    
    # Exit the position if the stock crosses the moving average
    elif (row['Close'] > row['MA'] and position == 'long') or (row['Close'] < row['MA'] and position == 'short'):
        # Calculate the profit/loss from the trade
        last_trade = trades[-1]
        if last_trade[0] == 'buy':
            profit_loss = num_shares * (row['Close'] - last_trade[2]) * last_trade[1]
        elif last_trade[0] == 'short':
            profit_loss = num_shares * (last_trade[2] - row['Close']) * last_trade[1]
        # Add the profit/loss to the initial capital
        initial_capital += profit_loss
        # Update the position
        position = None
        # Add the trade to the list of trades
        trades.append(('exit', profit_loss, row['Close'], initial_capital))
        
# Print the list of trades
for trade in trades:
    print(trade)


('buy', 257, 3.890000104904175, 9997.30026960373)
('exit', -1721.9001960754395, 3.2200000286102295, -1719.2004656791687)
('short', -66, 2.569999933242798, -1696.1999559402466)
('exit', -323.4000062942505, 2.0799999237060547, -3738.8004279136658)
('short', -141, 2.6500000953674316, -3736.5001344680786)
('exit', 112.79989242553711, 2.7300000190734863, -7362.500669956207)
('short', -175, 4.190000057220459, -7332.500100135803)
('exit', -979.9998998641968, 3.630000114440918, -15675.000669956207)
('buy', -486, 3.2200000286102295, -15649.200139045715)
('exit', -2041.2003707885742, 3.640000104904175, -2067.000901699066)
('short', -50, 4.119999885559082, -2059.999942779541)
('exit', -194.99993324279785, 3.7300000190734863, -4322.000777721405)
('short', -89, 4.829999923706055, -4298.699932098389)
('exit', 3551.099796295166, 8.819999694824219, -5069.600913524628)
('short', -45, 11.100000381469727, -4995.000171661377)
('exit', -805.4999828338623, 9.3100004196167, -10870.101068019867)
('short', -10

In [14]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf

# Download the data for the stock
stock = yf.download("RELIANCE.NS", start="2015-01-01", end="2022-03-20")

# Define the period for the strategy
period = 14

# Calculate the rolling mean, standard deviation, upper band, and lower band
stock['MA'] = stock['Close'].rolling(window=period).mean()
stock['SD'] = stock['Close'].rolling(window=period).std()
stock['Upper'] = stock['MA'] + (2 * stock['SD'])
stock['Lower'] = stock['MA'] - (2 * stock['SD'])

# Initialize the positions dataframe
positions = pd.DataFrame(index=stock.index).fillna(0.0)

# Initialize the portfolio dataframe with cash and stock columns
portfolio = pd.DataFrame(index=stock.index).fillna(0.0)
portfolio['Cash'] = 100000
portfolio['Stock'] = 0.0

# Initialize the breakout channels
stock['Green Channel'] = np.nan
stock['Red Channel'] = np.nan

# Implement the strategy for each row of the data
for i in range(period, len(stock)):
    # Determine if there is a bullish breakout
    if stock['Close'][i-2] > stock['Close'][i-3] and \
       stock['Close'][i-1] > stock['Close'][i-2] and \
       stock['Close'][i] > stock['Close'][i-1]:
        
        pivot_high = max(stock['Close'][i-2], stock['Close'][i-3])
        if stock['Close'][i] > pivot_high:
            stock['Green Channel'][i] = pivot_high
    
    # Determine if there is a bearish breakout
    elif stock['Close'][i-2] < stock['Close'][i-3] and \
         stock['Close'][i-1] < stock['Close'][i-2] and \
         stock['Close'][i] < stock['Close'][i-1]:
        
        pivot_low = min(stock['Close'][i-2], stock['Close'][i-3])
        if stock['Close'][i] < pivot_low:
            stock['Red Channel'][i] = pivot_low
    
    # Buy the stock when it crosses the lower bound
    if stock['Close'][i] < stock['Lower'][i] and portfolio['Cash'][i-1] > 0:
        portfolio['Stock'][i] = portfolio['Cash'][i-1] / stock['Close'][i]
        portfolio['Cash'][i] = 0.0
        
        positions.iloc[i] = 1.0
    
    # Short the stock when it crosses the upper bound
    elif stock['Close'][i] > stock['Upper'][i] and portfolio['Cash'][i-1] > 0:
        portfolio['Stock'][i] = -portfolio['Cash'][i-1] / stock['Close'][i]
        portfolio['Cash'][i] = 0.0
        
        positions.iloc[i] = -1.0
    
    # Exit the position when the stock price crosses the moving average
    elif (stock['Close'][i] > stock['MA'][i] and positions.iloc[i-1] == -1.0)


SyntaxError: invalid syntax (3273135887.py, line 65)